In [10]:
import h2o
from h2o.automl import H2OAutoML

In [11]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,18 hours 7 mins
H2O cluster timezone:,Europe/Vienna
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_mackenzie_dm3nzl
H2O cluster total nodes:,1
H2O cluster free memory:,1.776 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [12]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

In [14]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [15]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |██████████████████████████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_b8af285ab6dc1396f42d30553cc7455d> was cancelled by the user.

In [7]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20190626_151901,0.789972,0.551138,0.314771,0.432035,0.186654
StackedEnsemble_BestOfFamily_AutoML_20190626_151901,0.788199,0.553039,0.306089,0.432836,0.187347
XGBoost_grid_1_AutoML_20190626_151901_model_3,0.784588,0.558405,0.319401,0.435012,0.189235
XGBoost_1_AutoML_20190626_151901,0.782782,0.557031,0.331908,0.43506,0.189277
XGBoost_grid_1_AutoML_20190626_151901_model_4,0.782774,0.558375,0.316151,0.435543,0.189697
XGBoost_3_AutoML_20190626_151901,0.782434,0.563499,0.340872,0.437216,0.191158
XGBoost_2_AutoML_20190626_151901,0.781621,0.559814,0.326049,0.436018,0.190112
XGBoost_grid_1_AutoML_20190626_151901_model_1,0.781542,0.562387,0.32593,0.436869,0.190855
GBM_5_AutoML_20190626_151901,0.780837,0.559903,0.340848,0.436191,0.190263
GBM_2_AutoML_20190626_151901,0.780036,0.559806,0.339926,0.436415,0.190458


In [ ]:
# The leader model is stored here
aml.leader

In [9]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%
